# Guardian - Iteration 7

## Import Packages

In [1]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [2]:
iteration = 7

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2019-07-01",
    "to-date": "2019-12-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY2"),
}


In [3]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [4]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [5]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2019-07-01&to-date=2019-12-01&order-by=newest&api-key=934fc87f-0792-4733-b5ed-59d5aa136db0&page=1
status ok
total 33502
startIndex 1
pageSize 50
pages 671
orderBy newest
---- RUNTIME STATUS ---- 
(1/671) in 0:00:02.900673s
(2/671) in 0:00:15.717373s
(3/671) in 0:00:04.642533s
(4/671) in 0:00:06.124379s
(5/671) in 0:00:05.972499s
(6/671) in 0:00:03.827315s
(7/671) in 0:00:03.869364s
(8/671) in 0:00:03.565100s
(9/671) in 0:00:02.015096s
(10/671) in 0:00:02.406940s
(11/671) in 0:00:04.242003s
(12/671) in 0:00:05.999666s
(13/671) in 0:00:07.121868s
(14/671) in 0:00:06.551095s
(15/671) in 0:00:03.128969s
(16/671) in 0:00:04.135818s
(17/671) in 0:00:06.684608s
(18/671) in 0:00:06.936979s
(19/671) in 0:00:04.918392s
(20/671) in 0:00:04.094881s
(21/671) in 0:00:05.512747s
(22/671) in 0:00:17.586108s
(23/671) in 0:00:03.903089s
(24/671) in 0:00:02.892622s
(25/671) i

In [6]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}.csv.zip", compression="zip")
    return df

In [7]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(33502, 13) took 0:00:00.009644s

unfold_columns:
 shape=(33502, 48) took 0:00:08.491137s

drop_columns:
 shape=(33502, 30) took 0:00:00.018640s

create_csv:
 shape=(33502, 30) took 0:00:25.873213s

